## Data Challenge | Data cleaning and wrangling blueprint

The following is a blueprint to some of the data cleaning and wrangling you may want to do before starting to analyse and work with the provided dataset. Do not rely blindly on this code, instead be sure to check and double-check every step of your process. 

For instance, notice that the variable `B11_mese` in the `dataset_indagine_stati_aanimo.xlsx` is an **Excel serial date** (and not months as it may initially appear). In the "*Struttura dei dati*" tab, you can read that `B11_mese` is the "*Data e ora di compilazione del questionario*"; those are in fact the number of days since 01/01/1900. So, if you want to use this variable you will first need to convert it to a proper date (see a hint [here](https://stackoverflow.com/questions/68482380/strange-date-format-conversion)). 

Finally, if you decide to use this script, be sure to replace the path to your data files appropriately.

In [37]:
import pandas as pd
import numpy as np

## Loading and cleaning Indagine data

In [38]:
# Load Indagine data labels
indagine_label = pd.read_excel('data/dataset_indagine_stati_animo.xlsx', sheet_name='Labels')
print(indagine_label.shape)
indagine_label.head(3)

(8128, 108)


,ID_Rispondente,B11_Q62_A_1,B11_Q62_A_2,B11_Q62_A_3,B11_Q62_A_4,B11_Q62_A_5,B11_Q62_A_6,B11_Q62_A_7,B11_Q62_A_8,B11_Q62_A_9,...,p_Q14_A_2,p_Q14_A_3,p_Q14_A_4,p_Q14_A_5,p_Q14_A_6,p_Q14_A_7,p_Q14_A_8,p_Q14_A_9,Mese,peso
0,3936511661,Molto,Un po' sì e un po' no,Poco,Abbastanza,Poco,Molto,Un po' sì e un po' no,Molto,Abbastanza,...,5 volte,Due volte,3/ 4 volte,3/ 4 volte,Mai o meno di una volta,Due volte,Due volte,Mai o meno di una volta,2,0.581195
1,164860483,Un po' sì e un po' no,Per niente,Poco,Abbastanza,Poco,Abbastanza,Per niente,Abbastanza,Abbastanza,...,3/ 4 volte,Mai o meno di una volta,Due volte,Mai o meno di una volta,Una volta alla settimana,3/ 4 volte,3/ 4 volte,Mai o meno di una volta,2,5.458265
2,221250719,Molto,Per niente,Per niente,Abbastanza,Per niente,Abbastanza,Per niente,Abbastanza,Abbastanza,...,Mai o meno di una volta,Mai o meno di una volta,Più di 5 volte alla settimana,Mai o meno di una volta,Più di 5 volte alla settimana,3/ 4 volte,3/ 4 volte,Mai o meno di una volta,2,5.458265


In [39]:
# Load Indagine data values
indagine_value = pd.read_excel('data/dataset_indagine_stati_animo.xlsx', sheet_name='Values')
print(indagine_label.shape)
indagine_value.head(3)

(8128, 108)


,ID_Rispondente,B11_Q62_A_1,B11_Q62_A_2,B11_Q62_A_3,B11_Q62_A_4,B11_Q62_A_5,B11_Q62_A_6,B11_Q62_A_7,B11_Q62_A_8,B11_Q62_A_9,...,p_Q14_A_2,p_Q14_A_3,p_Q14_A_4,p_Q14_A_5,p_Q14_A_6,p_Q14_A_7,p_Q14_A_8,p_Q14_A_9,Mese,peso
0,3936511661,5,3,2,4,2,5,3,5,4,...,5.0,3.0,4.0,4.0,1.0,3.0,3.0,1.0,2,0.581195
1,164860483,3,1,2,4,2,4,1,4,4,...,4.0,1.0,3.0,1.0,2.0,4.0,4.0,1.0,2,5.458265
2,221250719,5,1,1,4,1,4,1,4,4,...,1.0,1.0,6.0,1.0,6.0,4.0,4.0,1.0,2,5.458265


In [40]:
# Load Indagine data structure --> indagine_lookup (data mapping)
indagine_lookup = pd.read_excel('data/dataset_indagine_stati_animo.xlsx', sheet_name='Struttura dei dati', skiprows=5, nrows=indagine_value.shape[1])
print(indagine_lookup.shape)

(108, 3)


In [41]:
# Cleaning of the Etichetta variable in the Indagine data structure DataFrame --> to generate the Domanda variable
indagine_lookup['Domanda'] = indagine_lookup['Etichetta'].apply(lambda st: st[st.find("Question: ")+10:np.min([st.find("]"), st.find("<span ")])] if st.find("Question: ") >= 0 else st)
indagine_lookup['Domanda'] = indagine_lookup['Domanda'].apply(lambda st: st[:st.find("<span ")] if st.find("<span ") >= 0 else st)
indagine_lookup['Domanda'] = indagine_lookup['Domanda'].apply(lambda st: st[:st.find("<br />")] if st.find("<br />") >= 0 else st)

In [42]:
# Cleaning of the Etichetta variable in the Indagine data structure DataFrame --> to generate the Topic variable
indagine_lookup['Topic'] = indagine_lookup['Etichetta'].apply(lambda st: st[st.find("Topic: ")+7:st.find("Question: ")-1] if st.find("Topic: ") >= 0 else st[:st.find("[Question: ")-1] if st.find("[Question: ") >= 0 else "")

In [43]:
for el in ['Age', 'Comune', 'Provincia', 'Regione', 'peso']: 
    indagine_lookup.loc[indagine_lookup['Variabile']==el,'Domanda'] = el

In [44]:
# Check top 3 rows of indagine_lookup DF
indagine_lookup.head(3)

,Variabile,Posizione,Etichetta,Domanda,Topic
0,ID_Rispondente,1,Codice identificativo della persona intervistata,Codice identificativo della persona intervistata,
1,B11_Q62_A_1,2,[Topic: Attivo/ intraprendente Question: Di se...,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
2,B11_Q62_A_2,3,[Topic: Turbato/ Agitato Question: Di seguito ...,Di seguito trovi una serie di parole che indic...,Turbato/ Agitato


## Loading and cleaning Profilo data

In [45]:
# Load Profilo data labels
profilo_label = pd.read_excel('data/dataset_profilo_intervistati.xlsx', sheet_name='Labels')
print(profilo_label.shape)
profilo_label.head(3)

(12853, 293)


,ID_Rispondente,B01_Q64_1,B01_Q64_2,B01_Q64_3,B01_Q64_4,B01_Q64_5,B01_Q64_6,B01_Q64_7,B01_Q64_8,B01_Q64_9,...,B10_Q66_7,B10_Q66_8,B10_Q66_9,B10_Q66_10,B10_Q66_11,B10_Q66_12,B10_Q66_13,B10_Q66_14,B10_Q66_15,B10_Q66_16
0,256848,Selected,Selected,Selected,Selected,Not Selected,Selected,Not Selected,Selected,Not Selected,...,NaN,Not Selected,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Selected
1,337988,Selected,Selected,Selected,Selected,Not Selected,Selected,Not Selected,Selected,Not Selected,...,Not Selected,Not Selected,Not Selected,Not Selected,Not Selected,NaN,Not Selected,Not Selected,Not Selected,Not Selected
2,369486,Selected,Selected,Selected,Selected,Not Selected,Selected,Not Selected,Selected,Not Selected,...,NaN,Selected,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Selected


In [46]:
# Load Profilo data values
profilo_value = pd.read_excel('data/dataset_profilo_intervistati.xlsx', sheet_name='Values')
print(profilo_label.shape)
profilo_value.head(3)

(12853, 293)


,ID_Rispondente,B01_Q64_1,B01_Q64_2,B01_Q64_3,B01_Q64_4,B01_Q64_5,B01_Q64_6,B01_Q64_7,B01_Q64_8,B01_Q64_9,...,B10_Q66_7,B10_Q66_8,B10_Q66_9,B10_Q66_10,B10_Q66_11,B10_Q66_12,B10_Q66_13,B10_Q66_14,B10_Q66_15,B10_Q66_16
0,256848,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,337988,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
2,369486,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [47]:
# Load Profilo data structure --> profilo_lookup (data mapping)
profilo_lookup = pd.read_excel('data/dataset_profilo_intervistati.xlsx', sheet_name='Struttura dei dati', skiprows=5, nrows=profilo_value.shape[1])
print(profilo_lookup.shape)

(293, 3)


In [48]:
# Cleaning of the Etichetta variable in the Profilo data structure DataFrame --> to generate the Domanda variable
profilo_lookup['Domanda'] = profilo_lookup['Etichetta'].apply(lambda st: st[st.find("Question: ")+10:np.min([st.find("]"), st.find("<br />")])] if st.find("Question: ") >= 0 else st)

In [49]:
# Cleaning of the Etichetta variable in the Profilo data structure DataFrame --> to generate the Topic variable
profilo_lookup['Topic'] = profilo_lookup['Etichetta'].apply(lambda st: st[st.find("/> ")+3:st.find(" [Question:")] if st.find("<img src=") >= 0 else st[0:st.find(" [Question:")] if st.find(" [Question:]") else "")
profilo_lookup['Topic'] = profilo_lookup['Topic'].apply(lambda st: st[st.find("[Topic: ")+8:st.find(" Question:")] if st.find("[Topic: ") >= 0 else st)
profilo_lookup['Topic'] = profilo_lookup['Topic'].apply(lambda st: st[0:st.find("<br />")] if st.find("<br />") >= 0 else st)

In [50]:
# Check top 3 rows of profilo_lookup DF
profilo_lookup.head(3)

,Variabile,Posizione,Etichetta,Domanda,Topic
0,ID_Rispondente,1,Codice identificativo della persona intervistata,Codice identificativo della persona intervistata,Codice identificativo della persona intervistat
1,B01_Q64_1,2,"<img src=""https://surveys.opline.it/media/logo...",Quali di questi strumenti di social networking...,Twitter
2,B01_Q64_2,3,"<img src=""https://surveys.opline.it/media/logo...",Quali di questi strumenti di social networking...,Facebook


## Melting Profilo data

In [51]:
# Melting the profilo_value DF --> transposing the data from horizontal to vertical
profilo_value_melt = pd.melt(profilo_value, id_vars='ID_Rispondente', var_name='domanda', value_name='values_risposta')
profilo_value_melt.shape

(3753076, 3)

In [52]:
# Melting the profilo_label DF --> transposing the data from horizontal to vertical
profilo_label_melt = pd.melt(profilo_label, id_vars='ID_Rispondente', var_name='domanda', value_name='labels_risposta')
profilo_label_melt.shape

(3753076, 3)

In [53]:
# Inner joining the profilo_label_melt and profilo_value_melt --> profilo_melt
profilo_melt = pd.merge(profilo_label_melt, profilo_value_melt, how='inner', on=['ID_Rispondente','domanda'])
print(profilo_melt.shape)
profilo_melt.head()

(3753076, 4)


,ID_Rispondente,domanda,labels_risposta,values_risposta
0,256848,B01_Q64_1,Selected,1.0
1,337988,B01_Q64_1,Selected,1.0
2,369486,B01_Q64_1,Selected,1.0
3,371674,B01_Q64_1,Selected,1.0
4,379221,B01_Q64_1,Selected,1.0


In [54]:
# Left joining the profilo_melt with the profilo_lookup (data mapping) DF --> generating the final profilo DF
profilo = pd.merge(profilo_melt, profilo_lookup[['Variabile', 'Domanda', 'Topic']], how='left', left_on='domanda', right_on='Variabile')
print(profilo.shape)
profilo.head()

(3753076, 7)


,ID_Rispondente,domanda,labels_risposta,values_risposta,Variabile,Domanda,Topic
0,256848,B01_Q64_1,Selected,1.0,B01_Q64_1,Quali di questi strumenti di social networking...,Twitter
1,337988,B01_Q64_1,Selected,1.0,B01_Q64_1,Quali di questi strumenti di social networking...,Twitter
2,369486,B01_Q64_1,Selected,1.0,B01_Q64_1,Quali di questi strumenti di social networking...,Twitter
3,371674,B01_Q64_1,Selected,1.0,B01_Q64_1,Quali di questi strumenti di social networking...,Twitter
4,379221,B01_Q64_1,Selected,1.0,B01_Q64_1,Quali di questi strumenti di social networking...,Twitter


In [55]:
# Renaming the variables and dropping unwanted columns
profilo.columns = ['idRispondente', 'cdDomanda', 'cdLabelsRisposta', 'cdValuesRisposta', 'Variabile', 'nmDomanda', 'nmRisposta']
profilo = profilo[['idRispondente', 'cdDomanda', 'cdLabelsRisposta', 'cdValuesRisposta', 'nmDomanda', 'nmRisposta']]
profilo.head()


,idRispondente,cdDomanda,cdLabelsRisposta,cdValuesRisposta,nmDomanda,nmRisposta
0,256848,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter
1,337988,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter
2,369486,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter
3,371674,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter
4,379221,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter


## Melting Indagine data

In [56]:
# Melting the indagine_value DF --> transposing the data from horizontal to vertical
indagine_value_melt = pd.melt(indagine_value, id_vars='ID_Rispondente', var_name='domanda', value_name='values_risposta')
indagine_value_melt.shape

(869696, 3)

In [57]:
# Melting the indagine_label DF --> transposing the data from horizontal to vertical
indagine_label_melt = pd.melt(indagine_label, id_vars='ID_Rispondente', var_name='domanda', value_name='labels_risposta')
indagine_label_melt.shape

(869696, 3)

In [58]:
# Inner joining the indagine_label_melt and indagine_value_melt --> indagine_melt
indagine_melt = pd.merge(indagine_label_melt, indagine_value_melt, how='inner', on=['ID_Rispondente','domanda'])
print(indagine_melt.shape)
indagine_melt.head()

(869696, 4)


,ID_Rispondente,domanda,labels_risposta,values_risposta
0,3936511661,B11_Q62_A_1,Molto,5
1,164860483,B11_Q62_A_1,Un po' sì e un po' no,3
2,221250719,B11_Q62_A_1,Molto,5
3,89508748,B11_Q62_A_1,Un po' sì e un po' no,3
4,98143773,B11_Q62_A_1,Poco,2


In [59]:
# Left joining the indagine_melt with the indagine_lookup (data mapping) DF --> generating the final indagine DF
indagine = pd.merge(indagine_melt, indagine_lookup[['Variabile', 'Domanda', 'Topic']], how='left', left_on='domanda', right_on='Variabile')
print(indagine.shape)
indagine.head()

(869696, 7)


,ID_Rispondente,domanda,labels_risposta,values_risposta,Variabile,Domanda,Topic
0,3936511661,B11_Q62_A_1,Molto,5,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
1,164860483,B11_Q62_A_1,Un po' sì e un po' no,3,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
2,221250719,B11_Q62_A_1,Molto,5,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
3,89508748,B11_Q62_A_1,Un po' sì e un po' no,3,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
4,98143773,B11_Q62_A_1,Poco,2,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente


In [60]:
# Renaming the variables and dropping unwanted columns
indagine.columns = ['idRispondente', 'cdDomanda', 'cdLabelsRisposta', 'cdValuesRisposta', 'Variabile', 'nmDomanda', 'nmRisposta']
indagine = indagine[['idRispondente', 'cdDomanda', 'cdLabelsRisposta', 'cdValuesRisposta', 'nmDomanda', 'nmRisposta']]
indagine.head()

,idRispondente,cdDomanda,cdLabelsRisposta,cdValuesRisposta,nmDomanda,nmRisposta
0,3936511661,B11_Q62_A_1,Molto,5,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
1,164860483,B11_Q62_A_1,Un po' sì e un po' no,3,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
2,221250719,B11_Q62_A_1,Molto,5,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
3,89508748,B11_Q62_A_1,Un po' sì e un po' no,3,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
4,98143773,B11_Q62_A_1,Poco,2,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente


## Save datasets to CSV files

In [61]:
# Check size and head of data 
print("Size:", indagine.shape)
indagine.head()

Size: (869696, 6)


,idRispondente,cdDomanda,cdLabelsRisposta,cdValuesRisposta,nmDomanda,nmRisposta
0,3936511661,B11_Q62_A_1,Molto,5,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
1,164860483,B11_Q62_A_1,Un po' sì e un po' no,3,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
2,221250719,B11_Q62_A_1,Molto,5,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
3,89508748,B11_Q62_A_1,Un po' sì e un po' no,3,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
4,98143773,B11_Q62_A_1,Poco,2,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente


In [62]:
# Check size and head of data 
print("Size:", profilo.shape)
profilo.head()

Size: (3753076, 6)


,idRispondente,cdDomanda,cdLabelsRisposta,cdValuesRisposta,nmDomanda,nmRisposta
0,256848,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter
1,337988,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter
2,369486,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter
3,371674,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter
4,379221,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter


In [105]:
# Save transposed and merged data to csv
profilo.to_csv('data_clean/profilo.csv', index=False)
indagine.to_csv('data_clean/indagine.csv', index=False)